In [ ]:
!pip install aitextgen==0.5.2
!pip install pytorch-lightning==1.3.8
!pip install torchmetrics==0.6.0
!pip install torchtext==0.8.1 #0.10.0
!pip install torchvision==0.8.2	

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.3/572.3 KB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.6/715.6 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
import os
import logging
import numpy as np
from aitextgen import aitextgen
from aitextgen.utils import build_gpt2_config
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
)


In [ ]:
!nvidia-smi -L # Figure out what type of GPU we are using.

GPU 0: Tesla T4 (UUID: GPU-f7845e36-7efb-1d12-828e-3a6fcb8bd333)


In [ ]:
!pip install -q transformers

In [ ]:
import torch
import shutil
from torch.utils.data import Dataset, random_split
from transformers import Trainer, TrainingArguments, GPTNeoForCausalLM, GPT2Tokenizer


from google.colab import drive


In [ ]:
!wget https://raw.githubusercontent.com/brunoklein99/deep-learning-notes/master/shakespeare.txt

--2023-03-28 18:15:25--  https://raw.githubusercontent.com/brunoklein99/deep-learning-notes/master/shakespeare.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94275 (92K) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>]  92.07K  --.-KB/s    in 0.002s  

2023-03-28 18:15:25 (37.4 MB/s) - ‘shakespeare.txt’ saved [94275/94275]



In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
shutil.copy("/content/shakespeare.txt","drive/MyDrive/AICamp/NLP")

'drive/MyDrive/AICamp/NLP/shakespeare.txt'

In [ ]:
def read_file(file_path):
    with open(file_path) as f:
        lines = [line for line in f]
        # lines.remove("")
    return lines


In [ ]:
file_path = "drive/MyDrive/AICamp/NLP/shakespeare.txt"

texts = read_file(file_path)
sonnets = []
sonnet = []
for text in texts:
  if len(text)>1:
    sonnet.append(text)
  else:
    sonnets.append(''.join(sonnet))
    sonnet = []


In [ ]:
datas = sonnets[2:-1]
print(len(datas))
for data in datas:
  if len(data)<1:
    datas.remove(data)
print(len(datas))

289
212


In [ ]:
class ShakespeareDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            # Encode the descriptions using the GPT-Neo tokenizer
            encodings_dict = tokenizer('<|startoftext|>' 
                                        + txt +    
                                        '<|endoftext|>',
                                        truncation=True,
                                        max_length=max_length, 
                                            padding="max_length")
            input_ids = torch.tensor(encodings_dict['input_ids'])    
            self.input_ids.append(input_ids)
            mask = torch.tensor(encodings_dict['attention_mask'])
            self.attn_masks.append(mask)
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
torch.manual_seed(42)

# Download the pre-trained GPT-Neo model's tokenizer
# Add the custom tokens denoting the beginning and the end 
# of the sequence and a special token for padding
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M",    
                            bos_token='<|startoftext|>',
                            eos_token='<|endoftext|>',
                            pad_token='<|pad|>')

# Download the pre-trained GPT-Neo model and transfer it to the GPU
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M").cuda()

# Resize the token embeddings because we've just added 3 new tokens 
model.resize_token_embeddings(len(tokenizer))

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/config.json
Model config GPTNeoConfig {
  "_name_or_path": "EleutherAI/gpt-neo-125M",
  

Embedding(50259, 768)

In [ ]:
max_length = max([len(tokenizer.encode(sonnet)) for sonnet in datas])

# Load dataset
dataset = ShakespeareDataset(sonnets, tokenizer, max_length)

# Split data into train/val
train_size = int(0.9 * len(dataset))

train_data, val_data = random_split(dataset, [train_size, len(dataset) - train_size])

max_length

351

In [ ]:
tokenizer.batch_decode(val_data[5])

["<|startoftext|> Betwixt mine eye and heart a league is took,\nAnd each doth good turns now unto the other,\nWhen that mine eye is famished for a look,\nOr heart in love with sighs himself doth smother;  \nWith my love's picture then my eye doth feast,\nAnd to the painted banquet bids my heart:\nAnother time mine eye is my heart's guest,\nAnd in his thoughts of love doth share a part.\nSo either by thy picture or my love,\nThy self away, art present still with me,\nFor thou not farther than my thoughts canst move,\nAnd I am still with them, and they with thee.\nOr if they sleep, thy picture in my sight\nAwakes my heart, to heart's and eye's delight.\n<|endoftext|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|

In [ ]:
training_args = TrainingArguments(output_dir=f'./results',
                                      num_train_epochs=5,
                                      logging_steps=1000,
                                      save_steps=5000,
                                      evaluation_strategy='steps',
                                      eval_steps=1000,                               
                                      per_device_train_batch_size=2,
                                      per_device_eval_batch_size=2,
                                      warmup_steps=100,
                                      learning_rate=5e-5,
                                      weight_decay=0.01,  
                                      logging_dir=f'./logs')

trainer = Trainer(model=model, args=training_args,  
                  train_dataset=train_data,
                  eval_dataset=val_data, 
                  # This custom collate function is necessary 
                  # to built batches of data
                  data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),       
              'attention_mask': torch.stack([f[1] for f in data]),
              'labels': torch.stack([f[0] for f in data])})

# Start training process!
trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 262
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 655
  Number of trainable parameters = 125200128


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=655, training_loss=1.055195908874046, metrics={'train_runtime': 252.4701, 'train_samples_per_second': 5.189, 'train_steps_per_second': 2.594, 'total_flos': 234581319198720.0, 'train_loss': 1.055195908874046, 'epoch': 5.0})

In [ ]:
trainer.save_model("drive/MyDrive/AICamp/NLP/")

Saving model checkpoint to drive/MyDrive/AICamp/NLP/
Configuration saved in drive/MyDrive/AICamp/NLP/config.json
Configuration saved in drive/MyDrive/AICamp/NLP/generation_config.json
Model weights saved in drive/MyDrive/AICamp/NLP/pytorch_model.bin


In [ ]:
tokenizer.save_pretrained("drive/MyDrive/AICamp/NLP/")

tokenizer config file saved in drive/MyDrive/AICamp/NLP/tokenizer_config.json
Special tokens file saved in drive/MyDrive/AICamp/NLP/special_tokens_map.json
added tokens file saved in drive/MyDrive/AICamp/NLP/added_tokens.json


('drive/MyDrive/AICamp/NLP/tokenizer_config.json',
 'drive/MyDrive/AICamp/NLP/special_tokens_map.json',
 'drive/MyDrive/AICamp/NLP/vocab.json',
 'drive/MyDrive/AICamp/NLP/merges.txt',
 'drive/MyDrive/AICamp/NLP/added_tokens.json')

In [ ]:
generated = tokenizer("<|startoftext|>", return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                                # bos_token='<|startoftext|>',
                                # eos_token='<|endoftext|>', pad_token='<|pad|>',
                                max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=20)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.27.3"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AttributeError: ignored

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from huggingface_hub import HfApi

api = HfApi()

In [ ]:

api.upload_folder(
    folder_path="drive/MyDrive/AICamp/NLP",
    path_in_repo = ".",
    repo_id="PriyamSheta/Shakespeare",
    repo_type="model",
    create_pr=1
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

'https://huggingface.co/PriyamSheta/Shakespeare/tree/refs%2Fpr%2F1/.'